In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
import glob
import json
import os
import pandas as pd
import pickle
import scipy
import math 
from mayavi import mlab
import shdom

from scipy.interpolate import interp1d
import scipy.io as sio
from scipy.optimize import curve_fit

rc = {
    "figure.figsize": (12,6),
    "xtick.major.size": 12.0,
    "xtick.minor.size": 8.0,
    "ytick.major.size": 12.0,
    "ytick.minor.size": 8.0,
    "axes.linewidth": 1.75,
    "xtick.color": '0',
    "ytick.color": '0',
    "axes.labelcolor": '0'
}


%matplotlib inline

ModuleNotFoundError: No module named 'vtkIOExodusPython'

# LOAD DATA FROM ESHKOL APRIL 2020

In [ ]:
dx,dy,dz=(1e-3*50,1e-3*50,1e-3*40) # in km
nz = 100
z_min = 0
z_max = 1e-3*40*nz
zgrid = np.linspace(z_min, z_max-dz ,nz)
# the above parameters should be known in advence.

"""
Eshkol said:
The matrices named fncd are 4D: 128x128x100x33, where horizontal resolution is 50 m and vertical is 40 m. 
The 33 are the size distribution bins according to the same rd.mat vector that I gave you in all previous files.
Number concentration = sum( fncd ) on the forth dimension 
effective radius - re(i,j,k) = sum( rd^3*fncd(i,j,k) )/ sum( rd^2*fncd(i,j,k) )

So from the nc files (each contains x,y,z,time,rn,rd,p,rho,FNCD) I need to extract:
'FNCD': <class 'netCDF4._netCDF4.Variable'>
'rd': <class 'netCDF4._netCDF4.Variable'>

Each nc file has:
variable units shape
x m (128,)
y m (128,)
z m (100,)
time d (1,)
rn micron (33,)
rd micron (33,)
p mb (100,)
rho g/m3 (100,)
FNCD #/cm3/bin  (1, 33, 100, 128, 128)

To work with nc files you need netcdf4 packeg. 
So in the terminal do:
conda install -c anaconda netcdf4
then use: from netCDF4 import Dataset
"""

import re
from tqdm import tqdm
from netCDF4 import Dataset

data_dir = '../WIZ_Clouds/April2020/'
format_ = 'BOMEX_128x128x100_2000CCN_50m_micro_128_*.com3D_int_2.nc'# lod one was .com3D_int_2.nc
volumes_paths = sorted(glob.glob(data_dir + '/'+format_))
volume_stamps = [re.findall('_(\d*).com3D_int_2.nc', i)[0] for i in volumes_paths]
volume_stamps = [int(i) for i in volume_stamps]# convert to integer 

SHOWVOL = True # if true, show evrey volume
IF_SAVE_txt = True
c=0
# for file in [volumes_paths[0]]:
for file in volumes_paths:
    nc = Dataset(file)
    rd = nc.variables['rd'][:].data # units micron
    FNCD = np.squeeze(nc.variables['FNCD'][:].data) # units #/cm3/bin
    # note that the shape of FNCD is now (33, 100, 128, 128)
    
    # LWC:
    # what Eshkol explained with old data: rho - density kg/m^3  - you can use this to convert mixing ratio (LWC [g/kg]]) to liquid water density [g/m^-3]]
    
    #current data:
    # rho - density g/m3:
    rho = nc.variables['rho'][:].data
    
    # loaded evrything I need, now calculate reff, veff, LWC 3d mat:
    min_diff = np.diff(np.squeeze(rd)).min()
    interp_bins = int(np.ceil((rd.max() - rd.min())/min_diff))
    r = np.linspace(rd.min(), rd.max(), interp_bins)
    
    S = np.sum(FNCD, axis=0)
    indxs = np.argwhere(S > 0)# cloudy voxel indexes
    
    # define all:
    SHAPE_3D = FNCD.shape[1:]
    RE = np.zeros(SHAPE_3D)
    VE = np.zeros(SHAPE_3D)
    LWC = np.zeros(SHAPE_3D)
    T = np.zeros(SHAPE_3D)
    # loop over all cloudy voxels
#     for vox_indx in tqdm(indxs):
    for vox_indx in indxs:
        DOPC = FNCD[:,vox_indx[0],vox_indx[1],vox_indx[2]] # distribution of droplets consintration 
        DOPC_interpulated = interp1d(np.squeeze(rd),DOPC)(r)
        #---------------------------------------------
        # calculate reff, veff, LWC from the FNCD (here it doesn't assume gamma distribution):
        reff = np.trapz(DOPC_interpulated*r**3,r, axis=0)/np.trapz(DOPC_interpulated*r**2,r, axis=0)
        RE[vox_indx[0],vox_indx[1],vox_indx[2]] = reff
        VE[vox_indx[0],vox_indx[1],vox_indx[2]] = np.trapz(((r-reff)**2)*(r**2)*DOPC_interpulated,r, axis=0)/(reff**2*np.trapz(DOPC_interpulated*r**2,r, axis=0))

        LWC[vox_indx[0],vox_indx[1],vox_indx[2]] = (10e-3)*(4/3)*np.pi*rho[vox_indx[0]]*np.trapz((r**3)*DOPC_interpulated,r, axis=0)
        # units of rho are g/m^3, of DOPC are #/cm3/um, of r um. The units of the result LWC are [g/m^3]
#         c = c + 1
        T[vox_indx[0],vox_indx[1],vox_indx[2]] = S[vox_indx[0],vox_indx[1],vox_indx[2]]
#         if(c>10000):
#             break;

    
    # here the 3d shape is (100, 128, 128)
    RE = np.transpose(RE, (2, 1, 0))
    VE = np.transpose(RE, (2, 1, 0))
    LWC = np.transpose(RE, (2, 1, 0))
    # here the 3d shape is (128, 128,100)
    
    if(IF_SAVE_txt):
        comment_line = "Data from Eshkol Itan"
        original_name = os.path.basename(file)
        file_name = original_name.split('.')[0]+'.txt'
        print('saving {}'.format(file_name))

        np.savetxt(file_name, X=np.array([RE.shape]), fmt='%d', header=comment_line)
        f = open(file_name, 'ab') 
        
        
        np.savetxt(f, X=np.concatenate((np.array([dx, dy]), zgrid)).reshape(1,-1), fmt='%2.3f')
        nx, ny, nz = RE.shape
        lwc = LWC
        reff = RE
        veff = VE
        y, x, z = np.meshgrid(range(ny), range(nx), range(nz))
        data = np.vstack((x.ravel(), y.ravel(), z.ravel(), lwc.ravel(), reff.ravel(), veff.ravel())).T
        np.savetxt(f, X=data, fmt='%d %d %d %.5f %.3f')        
        f.close()  
        
    if(SHOWVOL):
        res3d = (0.05,0.05,0.04) # horizontal resolution is 50 m and vertical is 40 m
        viz3D(RE, res3d = res3d,title='re [um]')
            
            
# in pyshdom liquid water content lwc is of units (g/m^3).

#  Work on one voxel to test the units and some operations


In [ ]:
import re
from tqdm import tqdm
from netCDF4 import Dataset

data_dir = '../WIZ_Clouds/April2020/'
format_ = 'BOMEX_128x128x100_2000CCN_50m_micro_128_*.com3D_int_2.nc'# lod one was .com3D_int_2.nc
volumes_paths = sorted(glob.glob(data_dir + '/'+format_))
volume_stamps = [re.findall('_(\d*).com3D_int_2.nc', i)[0] for i in volumes_paths]
volume_stamps = [int(i) for i in volume_stamps]# convert to integer 


nc = Dataset(volumes_paths[0])
rd = nc.variables['rd'][:].data # units micron
FNCD = np.squeeze(nc.variables['FNCD'][:].data) # units #/cm3/bin
# note that the shape of FNCD is now (33, 100, 128, 128)

# LWC:
# what Eshkol explained with old data: rho - density kg/m^3  - you can use this to convert mixing ratio (LWC [g/kg]]) to liquid water density [g/m^-3]]

#current data:
# rho - density g/m3:
rho = nc.variables['rho'][:].data

# from plt.plot(rd) - you can see that the bins are not linear
min_diff = np.diff(np.squeeze(rd)).min()
interp_bins = int(np.ceil((rd.max() - rd.min())/min_diff))
r = np.linspace(rd.min(), rd.max(), interp_bins)

# here, take for example one voxel:
S = np.sum(FNCD, axis=0)
indxs = np.argwhere(S > 0)# cloudy voxel indexes
example_vox_indx = indxs[1166]
# maximum radius is here ?
# minimum radius is here ?

DOPC = FNCD[:,example_vox_indx[0],example_vox_indx[1],example_vox_indx[2]] # distribution of droplets consintration 
DOPC_interpulated = interp1d(np.squeeze(rd),DOPC)(r)
#---------------------------------------------
# calculate reff, veff, LWC from the FNCD (here it doesn't assume gamma distribution):
reff = np.trapz(DOPC_interpulated*r**3,r, axis=0)/np.trapz(DOPC_interpulated*r**2,r, axis=0)
veff = np.trapz(((r-reff)**2)*(r**2)*DOPC_interpulated,r, axis=0)/(reff**2*np.trapz(DOPC_interpulated*r**2,r, axis=0))

LWC = (10e-3)*(4/3)*np.pi*rho[example_vox_indx[0]]*np.trapz((r**3)*DOPC_interpulated,r, axis=0)
# units of rho are g/m^3, of DOPC are #/cm3/um, of r um. The units of the result LWC are [g/m^3]
print("calculated LWC = {} [g/m^3]".format(LWC))
print("calculated reff = {} [um]".format(reff))
print("calculated veff = {}".format(veff))

# try to fit gamma distribution:
NC = np.trapz(DOPC,np.squeeze(rd), axis=0)# total consintration
test_gamma = gamma_distribution(r,NC,reff,veff)  

#---------------------------------------------
re_test = np.trapz(test_gamma*r**3,r, axis=0)/np.trapz(test_gamma*r**2,r, axis=0)
print("gamma reff = {}".format(re_test))

# plot
f, ax = plt.subplots(1, 1, figsize=(8, 8))
plt.plot(r,DOPC_interpulated,label='Original')
plt.xlim([0 ,500])
plt.xlabel('radius [um]', fontsize=16)
plt.ylabel('distribution dn(r)/dr [#/cm^3/um]', fontsize=16)
plt.title('one voxel distribution example', fontsize=16)

# gamma distribution test:
plt.plot(r,test_gamma,linewidth=2.0,label='Gamma',linestyle = '--')
LWC = 0.0
for ri,n in zip(r,test_gamma):
    LWC = LWC + (1.0E-6*1*n*(4*math.pi/3)*ri**3)
    
# gamma distribution of pyshdom:
mie_mono = shdom.MieMonodisperse(particle_type='Water')
mie_mono.set_wavelength_integration(wavelength_band=(wavelengths[0], wavelengths[0]))   
mie_mono.set_radius_integration(minimum_effective_radius=rd.min(), max_integration_radius=250)

size_distribution = shdom.SizeDistribution(type='gamma')
size_distribution.set_parameters(reff=np.linspace(1, 1.5*reff, 100), veff=np.linspace(0.01, 0.5, 50))
size_distribution.compute_nd(radii=r, particle_density=mie_mono.pardens)
#
size_dist = size_distribution.get_nd(reff=reff, veff=veff)

#---------------------------------------------
re_test = np.trapz(size_dist*r**3,r, axis=0)/np.trapz(size_dist*r**2,r, axis=0)
print("shdom reff = {}".format(re_test))

# plt.plot(r,LWC*size_dist,label='shdom v_eff= {}'.format(veff),linestyle = '--')

ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
plt.grid(True)
plt.legend(prop={'size': 16})